# Convert xlsx file to txt file

In [1]:
import re
import pandas as pd
from rdflib import Graph, Literal, Namespace, RDF, URIRef
from rdflib.namespace import SKOS, DCTERMS

In [2]:
with open('sachgebietssystematik.txt', 'w') as file:
    pd.read_excel('2019-03-29-Suchbegriff_Sachgebietssystematik-AG-MuD-Version.xlsx').to_string(file, index=False)

In [3]:
class Node:
    def __init__(self, _id=None, prefLabel=None, note=None):
        self.id = _id
        self.prefLabel = prefLabel
        self.note = note
        self.children = []
    def __repr__(self):
        return self.prefLabel

In [4]:
txt = open('sachgebietssystematik.txt').readlines()
txt_clean = []
for line in txt:
    line = line.replace('\n', '')
    if line == '':
        continue
    txt_clean.append(line)

In [6]:
data = []
text_id = re.compile(r"\d\d\d(\s\d\d)*[^\d\d.]")
text_label = re.compile(r"(\d\d\d(\s\d\d)*[^\d\d.])(([\u00C0-\u017FA-Za-z0-9,;'.-]+)\s)+")
# re.search(text_label, string).group()
text_note = re.compile(r"(\s[\u00C0-\u017FA-Za-z0-9,.;'-]+)+(\s)*$", re.DOTALL)


for i, item in enumerate(txt_clean[1:]):
    try:
        d = {}
        
        d['id'] = re.search(text_id, item).group().strip()
        d['prefLabel'] = re.sub(re.search(text_id, item).group(), '', re.search(text_label, item).group()).strip()
        d['note'] = re.search(text_note, item).group().split(";")
        #d['note'] = ''
        level = len(d['id'].split(' ')) -1
        d['level'] = level
        d['id'] = d['id'].replace(' ', '')
        data.append(d)
    except AttributeError:
        print(i)
        print(f"Error parsing: {d['id']}")
        print(f"Error parsing: {d['note']}")

In [7]:
root = Node()

try:
    for record in data:
        last = root
        for _ in range(record['level']):
            last = last.children[-1]
        last.children.append(Node(record['id'], record['prefLabel'], record['note']))
except IndexError:
    pass


In [8]:
def print_tree(root, depth=0):
    for child in root.children:
        print('  ' * depth + '%r' % child)
        print_tree(child, depth + 1)

In [8]:
print(print_tree(root))

ARBEITSLEHRE
  Hinführung zur Arbeitswelt
    Arbeitsbedingungen, Arbeitsabläufe
    Arbeitswert
    Arbeitssicherheit
    Recht im Betrieb
    Unternehmensformen, Unternehmensstrukturen
    Strukturwandel, Rationalisierung, Humanisierung
    Gesellschaftliche Bedingungen
    Geschichte von Arbeit und Technik
  Berufskunde
    Berufswahlkriterien
    Berufsbilder
      Berufe im naturwissenschaftlich-technischen Bereich
        Metallberufe
        Elektroberufe
        Berufsbereiche Bau, Bergbau, Steine
        Berufsbereiche Textil, Bekleidung, Leder
        Berufsbereiche Druck und Papier
        Berufsbereiche Holz und Kunststoff
        Berufsbereiche Chemie, Glas, Keramik
      Berufe im medizinischen Bereich
      Berufe im sozialen Bereich
      Berufe im hauswirtschaftlichen Bereich
      Berufe im pädagogischen Bereich
      Berufe im kaufmännischen Bereich
      Berufe in der Datenverarbeitung
      Berufe im Behördendienst
      Berufe im Verkehrswesen
      Berufe bei der

In [9]:
name_systematik = 'eaf-sachgebietssystematik'
filename = 'eaf-sachgebietssystematik'
g = Graph()
n = Namespace("http://w3id.org/openeduhub/vocabs/" + name_systematik + "/")

category = URIRef(n)

title = Literal(name_systematik, lang="de")
description = Literal(name_systematik, lang="de")
creator = Literal("<https://creator.com>")

# Add triples using store's add method.
g.add( (category, RDF.type, SKOS.ConceptScheme) )
g.add( (category, DCTERMS.title, title) )
g.add( (category, DCTERMS.description, description) )
g.add( (category, DCTERMS.creator, creator) )

# define relevant predicates

narrower = 'http://www.w3.org/2004/02/skos/core#narrower'
broader = 'http://www.w3.org/2004/02/skos/core#broader'
topConceptOf = 'http://www.w3.org/2004/02/skos/core#topConceptOf'
note = 'http://www.w3.org/2004/02/skos/core#note'


def add_items(root):
    for item in root.children:

        node = n + URIRef(item.id)
        node_prefLabel = Literal(item.prefLabel, lang="de")
        

        g.add( (node, RDF.type, SKOS.Concept) )
        g.add( (node, SKOS.prefLabel, node_prefLabel))
        
        for note in item.note:
            node_note = Literal(note.strip(), lang="de")
            g.add( (node, SKOS.note, node_note))
        
        g.add( (node, SKOS.inScheme, category))
        
        if item.children != []:
            for child in item.children:
                g.add( (node, SKOS.narrower, n + URIRef(child.id)))
                g.add( (n + URIRef(child.id), SKOS.broader, node))

        add_items(item)
            
add_items(root)

for child in root.children:
    node = n + URIRef(child.id)
    g.add( (category, SKOS.hasTopConcept, node))
    g.add( (node, SKOS.topConceptOf, category ))

# Bind a few prefix, namespace pairs for more readable output
g.bind("dct", DCTERMS)
g.bind("skos", SKOS)

output = g.serialize(format='turtle', base=n).decode("utf-8")

with open(filename + '.ttl', 'w') as f:
    f.write(output)
    f.close()

In [10]:
total_number = 0
for s,p,o in g.triples((None, None, SKOS.Concept)):
    total_number += 1
print(total_number)

len(txt_clean[1:])

1619


1619